In [21]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem(object):
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity

class Order(object):
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total()-discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due:{:.2f}>'
        return fmt.format(self.total(), self.due())
    
class Promotion(ABC):
    
    @abstractmethod
    def discount(self, order):
        """返回折扣金额"""
    
class FidelityPromo(Promotion):
    """为积分为1000或以上的顾客提供5%折扣"""
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
    
class BulkItemPromo(Promotion):
    """单个商品为20个或以上的提供10%的折扣"""
    def discount(self, order):
        discount = 0 
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() *0.1
        return discount

class LargeOrderPromo(Promotion):
    """订单中不同商品达到10个或以上的提供7%的折扣"""
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items)>= 10:
            return order.total() *0.07
        return 0

In [22]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 0.5),
       LineItem('apple', 4, 1.5),
        LineItem('watermellon',5, 5.0)]



In [23]:
Order(joe, cart, FidelityPromo())

<Order total: 33.00 due:33.00>

In [26]:
Order(ann, cart, FidelityPromo())

<Order total: 33.00 due:31.35>

In [27]:
banana_cart = [LineItem('banana', 30, 5), LineItem('apple', 10, 1.5)]

In [29]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 165.00 due:150.00>